In [132]:
import qdrant_client

client = qdrant_client.QdrantClient("http://localhost:6333", prefer_grpc=True)
client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='knowledge-base'), CollectionDescription(name='KB-json-f'), CollectionDescription(name='KB-json-a'), CollectionDescription(name='knowledge-base-json'), CollectionDescription(name='KB-json-d'), CollectionDescription(name='KB-json-b'), CollectionDescription(name='KB-json-c'), CollectionDescription(name='KB-json-e')])

In [133]:
import pandas as pd

df = pd.read_csv("../output.csv")

In [134]:
df = df.head(1000)

In [135]:
COLLECTION_NAME="KB-json-f"

In [136]:
docs = df.to_json(orient='records', lines=True)

In [137]:
docs = docs.split("\n")

In [138]:
client.add(collection_name=COLLECTION_NAME, documents=docs)

['e949e8f4f54d4e93ac1bfe6e0e502288',
 '7a6d91e06c5c43ebb327c97ba3ff765c',
 'bf51ccf959a1458781edd6d8628c7b44',
 '6af02a738cb14f70a6f31599e4f30f5a',
 '12cdf90a76464f6a99bd2ed18f8e576f',
 'a9f817b05aac4c0787d6cad355c9f7af',
 'c19bb76d80b747b891a2f37aef0552f7',
 '90eb67327d6f409dbc215ce44cefe83b',
 '26129acedf5144a2aed37e86ded4d209',
 '176319f7e2ad4d11956922ef02c8d74d',
 '386c2e75f7a0414294c0a691f4473c7a',
 '9a40d3800c8e428386d917934d8f064c',
 'fc34c4a027c94115a4f3e1610b156500',
 '4ec9c88cb10d4389899654fcdfdb746a',
 '6af028da1cd44b17bcce29e691b743ee',
 '46d86f185f5440a29d968181332996bb',
 '4595c9f7cbe94e0faad3c2b5732bfbff',
 '987a76b2f88d4195b6d4268366abd742',
 '65cd9d15d65c496a88e9015647aaee46',
 'c38bd127110e4ba08ddcb700e55eb11d',
 'cde3ba349bcf458d87743e4c4026275c',
 'b6bd98c6c23a4c9aa0df51134d9e58fa',
 '629d8e3010a24187a49d9b583f0cf6bb',
 '4b3fbfdde35f498eb8e70e056364b642',
 '6e3ece323c224b46b72b7fc7c762280b',
 '21a3fbd56c7649dbad488bb7ba4b25fa',
 '571f2d02394e4df180b806526891eade',
 

In [139]:
results = client.query(
    collection_name=COLLECTION_NAME,
    query_text="commit",
    limit=100,
)
print(results)

[QueryResponse(id='99836e8c-146d-4ae9-aec5-623ed93e94dc', embedding=None, metadata={'document': '{"filename":".pre-commit-config.yaml","message":"\\ud83d\\udcdd Update release notes\\n","author_email":"github-actions@github.com","hash":"e7756ae7dcffa0b73767d354ca158fd6b7bc87ed","bug_spot_likelihood":null,"commit_timestamp":"2023-12-20 17:06:01 +0000 UTC"}'}, document='{"filename":".pre-commit-config.yaml","message":"\\ud83d\\udcdd Update release notes\\n","author_email":"github-actions@github.com","hash":"e7756ae7dcffa0b73767d354ca158fd6b7bc87ed","bug_spot_likelihood":null,"commit_timestamp":"2023-12-20 17:06:01 +0000 UTC"}', score=0.891374409198761), QueryResponse(id='e58a9187-d5bf-4322-82fd-2d7a39e2077d', embedding=None, metadata={'document': '{"filename":".pre-commit-config.yaml","message":"\\ud83d\\udd16 Release version 0.105.0\\n","author_email":"tiangolo@gmail.com","hash":"d8185efb6effd36c162bc7ab8e1e2c597fb3b7a8","bug_spot_likelihood":null,"commit_timestamp":"2023-12-12 00:32:48

In [140]:
import os

# Fill the environmental variable with your own OpenAI API key
# See: https://platform.openai.com/account/api-keys
os.environ["OPENAI_API_KEY"] = "x"

In [141]:
from openai import OpenAI

def rag(question: str, n_points: int = 30) -> str:
    results = client.query(
        collection_name=COLLECTION_NAME,
        query_text=question,
        limit=n_points,
    )

    context = "\n".join(r.document for r in results)

    metaprompt = f"""
    You are a software architect. 
    Answer the following question using the provided context. 
    If you can't find the answer, do not pretend you know it, but answer "I don't know".
    
    Question: {question.strip()}
    
    Context: you have the following list of Git Commits: {context.strip()}.
    
    Answer:
    """
    oaclient = OpenAI()
    print(context)

    completion = oaclient.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "user", "content": metaprompt},
        ],
        timeout=10.0,
    )
    return completion.choices[0].message.content

In [142]:
rag("Give me a summary of the context")



{"filename":"docs\/em\/docs\/tutorial\/sql-databases.md","message":"\ud83d\udcdd Update release notes\n","author_email":"tiangolo@gmail.com","hash":"36c26677682c4245183912e00fc057c05cc6cf7a","bug_spot_likelihood":null,"commit_timestamp":"2023-12-12 00:34:36 +0000 UTC"}
{"filename":"docs\/em\/docs\/tutorial\/query-params.md","message":"\ud83d\udcdd Update release notes\n","author_email":"tiangolo@gmail.com","hash":"36c26677682c4245183912e00fc057c05cc6cf7a","bug_spot_likelihood":null,"commit_timestamp":"2023-12-12 00:34:36 +0000 UTC"}
{"filename":"docs\/em\/docs\/tutorial\/query-params.md","message":"\ud83d\udd16 Release version 0.105.0\n","author_email":"tiangolo@gmail.com","hash":"d8185efb6effd36c162bc7ab8e1e2c597fb3b7a8","bug_spot_likelihood":null,"commit_timestamp":"2023-12-12 00:32:48 +0000 UTC"}
{"filename":"docs\/em\/docs\/tutorial\/sql-databases.md","message":"\ud83d\udcdd Update release notes\n","author_email":"github-actions@github.com","hash":"e7756ae7dcffa0b73767d354ca158fd

'The context provides a list of Git commits. Each commit is documented with its file name, the commit message, the author\'s email, the hash of the commit, the likelihood of the commit being a bug spot, and the timestamp of the commit. The author of many of these commits is "tiangolo@gmail.com" and most of the commit messages indicate an update to the release notes.'